In [ ]:
from ..EA.individual_elitist import IndividualElitist
from numpy.random import multivariate_normal
from numpy import exp, sqrt, outer, zeros, identity
from copy import deepcopy

class IndividualCMAES(IndividualElitist):

    def __init__(self, problem, success_rate, sigma):
        super().__init__(problem)
        self.parent = None
        self.success_rate = success_rate
        self.sigma = sigma
        self.evolution_path = zeros(problem.d)
        self.C = identity(problem.d)

    def set_parent(self, parent):
        self.parent = parent

    def clone(self):
        child = IndividualCMAES(self.problem, self.success_rate, self.sigma)
        child.coordinates = deepcopy(self.coordinates)
        child.objectives = deepcopy(self.objectives)
        child.parent = self
        return child

    def mutate(self):
        self.coordinates = multivariate_normal(self.coordinates, (self.sigma**2)*self.C)
        self.problem.check_coordinates(self.coordinates)
        self.evaluate_objectives()
        self.reset_sorting_attributes()

    def update_step_size(self, success_rate, target_success_rate, delta, l):
        learning_rate = (l*target_success_rate) / (2 + l*target_success_rate)
        self.success_rate = (1-learning_rate)*self.success_rate + learning_rate*success_rate
        self.sigma = self.sigma * exp( (self.success_rate-target_success_rate) / (delta * (1-target_success_rate)) )

    def update_covariance(self, step, treshold, evolution_rate, covariance_rate):
        if self.success_rate < treshold:
            self.evolution_path = (1-evolution_rate) * self.evolution_path + sqrt(evolution_rate*(2-evolution_rate)) * step
            self.C = (1-covariance_rate) * self.C + covariance_rate * outer(self.evolution_path, self.evolution_path)
        else:
            self.evolution_path = (1-evolution_rate) * self.evolution_path
            self.C = (1-covariance_rate) * self.C + covariance_rate * ( outer(self.evolution_path, self.evolution_path) + evolution_rate*(2-evolution_rate) * self.C )

    def __str__(self):
        print("Rank:", self.rank, "\tCrowding distance:", self.crowding_distance, "\tParent:", self.parent.__repr__())
        print("Success rate:", self.success_rate, "\tSigma: ", self.sigma)
        print("Evolution path [0:3]:", self.evolution_path[0:3], "\nC [0:3,0:3]: ")
        print(self.C[0:3,0:3])
        print("f", self.coordinates, " = ", self.objectives)